In [1]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
X_train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/X_train.csv")
X_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/X_test.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/y_train.csv")

In [7]:
# 데이터 크기 확인
X_train.shape, X_test.shape, y_train.shape

((29304, 15), (3257, 15), (29304, 2))

In [6]:
# 데이터 샘플
X_train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,3331,34.0,State-gov,177331,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,Black,Male,4386,0,40.0,United-States
1,19749,58.0,Private,290661,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40.0,United-States
2,1157,48.0,Private,125933,Some-college,10,Widowed,Exec-managerial,Unmarried,Black,Female,0,1669,38.0,United-States
3,693,58.0,Private,100313,Some-college,10,Married-civ-spouse,Protective-serv,Husband,White,Male,0,1902,40.0,United-States
4,12522,41.0,Private,195661,Some-college,10,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,54.0,United-States


In [10]:
# 타겟 수 확인
y_train['income'].value_counts()

,count
income,
<=50K,22263
>50K,7041


In [11]:
# type 확인
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 3.4+ MB


In [14]:
# 수치형 데이터
cols = ['age', 'fnlwgt', 'education.num', 'capital.gain', 'capital.loss', 'hours.per.week']

In [16]:
# 수치형 데이터 통계
X_train[cols].describe()

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29292.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29291.000000
mean,38.553223,1.897488e+05,10.080842,1093.858722,86.744506,40.434229
std,13.628811,1.055250e+05,2.570824,7477.435640,401.518928,12.324036
min,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [22]:
# 결측값
X_train[cols].isnull().sum()

,0
age,0
fnlwgt,0
education.num,0
capital.gain,0
capital.loss,0
hours.per.week,0


In [23]:
# 결측치 처리
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [24]:
# 결측값 확인
X_train[cols].isnull().sum()

,0
age,0
fnlwgt,0
education.num,0
capital.gain,0
capital.loss,0
hours.per.week,0


In [25]:
# target값 변경
# <=50K -> 0
# >50K -> 1
# y = (y_train['income'] != '<=50K').astype(int)

In [29]:
y=(y_train['income'] == '>50K').astype(int)
y.head()

,income
0,1
1,0
2,0
3,1
4,0


In [32]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train[cols], y)
pred = rf.predict(X_test[cols])

In [38]:
# 예측 및 csv 파일 생성
pred[:20]
submit = pd.DataFrame({
    'id':X_test['id'],
    'income':pred
})
submit.to_csv("00000.csv", index=False)

In [40]:
# 평가
from sklearn.metrics import accuracy_score
y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/y_test.csv")
ans=(y_test['income'] == '>50K').astype(int)
accuracy_score(ans, pred)

0.8096407737181456

In [45]:
# 검증용 데이터 분리
from sklearn.model_selection import train_test_split
y=(y_train['income'] == '>50K').astype(int)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y, test_size=0.1, random_state=2022)

In [46]:
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((26373, 15), (2931, 15), (26373,), (2931,))

In [56]:
# 의사결정나무
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_tr[cols], y_tr)
pred = dt.predict_proba(X_val[cols])
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred[:, 1])

0.7019568638271227

In [57]:
# 랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_tr[cols], y_tr)
pred = rf.predict_proba(X_val[cols])
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred[:, 1])

0.846184228927366

In [58]:
# XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_tr[cols], y_tr)
pred = xgb.predict_proba(X_val[cols])
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred[:, 1])

0.8859038904196574

In [62]:
# 예측 및 csv 파일 생성

from xgboost import XGBClassifier
xgb = XGBClassifier()
y_train=(y_train['income'] == '>50K').astype(int)
xgb.fit(X_train[cols], y_train)
pred = xgb.predict_proba(X_test[cols])

submit = pd.DataFrame({
    'id':X_test['id'],
    'income':pred[:, 1]
})
submit.to_csv("22222.csv", index=False)

In [63]:
from sklearn.metrics import roc_auc_score
y_test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/main/p2/data_atype_y/y_test.csv")
ans = (y_test['income'] != '<=50K').astype(int)
roc_auc_score(ans, pred[:,1])

0.8798003154253154